In [1]:
import torch
import matplotlib.pyplot as plt
import os
import sys

from PIL import Image
import torchvision.transforms as transforms
from conv_autoenc import ConvAE

device = 'cpu'

In [2]:
run_id = '1717378265'

model = ConvAE(filter_sizes=[2048,1536,1024,512]).to(device)
model.load_state_dict(torch.load(f'output/ae/{run_id}/saved_model/best_cae.pt', map_location='cpu'))

<All keys matched successfully>

In [3]:
import os
import csv

try:
    os.makedirs('processed')
except:
    pass

In [12]:
t_steps = 400

def crop_left(im): 
    return transforms.functional.crop(im, top=0, left=0, height=420, width=560)

img_tf = transforms.Compose([
    transforms.Lambda(crop_left),  # Crop the left side
    transforms.Lambda(lambda x: x.convert('RGB')),  # Ensure the image is in RGB mode
    transforms.Resize(size=(128, 96), antialias=True),  # Downsample to 128xH
    transforms.Pad(padding=(16, 0, 16, 0)), # Pad to 128x128
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to range [-1, 1]
])

for i in range(24):
    data_folder = f'/home/yigit/projects/cnep/baxter/data/ral/{i}/'
    # iterate over all files in the in_folder
    for filename in os.listdir(data_folder):
        d = os.path.join(data_folder, filename)
        if filename.endswith('.jpeg'):
            img = img_tf(Image.open(d))
            with torch.no_grad():
                encoding = model.encode(img)
                torch.save(data, f"processed/enc_{i}.pt")

        if filename.endswith('.csv'):
            temp_data = []
            with open(d, 'r') as f:
                for j, line in enumerate(csv.reader(f)):
                    if j > 0:
                        temp_data.append([float(line[3]), float(line[4]), float(line[5]), float(line[6]), float(line[7]), float(line[8]), float(line[9]), float(line[10])])  # p, q, gripper

            ids = torch.linspace(0, len(temp_data)-1, t_steps).int().tolist()
            data = torch.tensor([temp_data[i] for i in ids])
            torch.save(data, f"processed/traj_{i}.pt")